In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve().parent
sys.path.append(str(PROJECT_ROOT))


from src.rag.vector_store import ComplaintVectorStore
from src.rag.retriever import ComplaintRetriever
from src.rag.generator import ComplaintGenerator
from src.rag.pipeline import ComplaintRAGPipeline
from src.rag.evaluate import evaluate


c:\Users\amare\Documents\KAIM\week-7\rag-complaint-chatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Initialize vector store
vector_store = ComplaintVectorStore(
    parquet_path="../data/complaint_embeddings.parquet"
)

# Build FAISS index
vector_store.load_or_build()



Loading existing FAISS index...


<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000001E689E37DE0> >

In [3]:
# Retriever using the vector store
retriever = ComplaintRetriever(
    vector_store=vector_store
)

# Generator
generator = ComplaintGenerator()


Device set to use cpu


✓ ComplaintGenerator loaded: google/flan-t5-base


In [4]:
# Connect retriever + generator
rag_pipeline = ComplaintRAGPipeline(
    retriever=retriever,
    generator=generator
)


In [5]:
question = "What problems do customers have with credit cards?"

result = rag_pipeline.answer(question)

print("Question:", result["question"])
print("Answer:\n", result["answer"])
print("\nSample Source Metadata:", result["sources"][0]["metadata"])


Question: What problems do customers have with credit cards?
Answer:
 lack of service and clarity

Sample Source Metadata: {'chunk_index': 2, 'company': 'CITIBANK, N.A.', 'complaint_id': '6063390', 'date_received': '2022-10-07', 'issue': 'Other features, terms, or problems', 'product': 'Credit card or prepaid card', 'product_category': 'Credit Card', 'state': 'AZ', 'sub_issue': 'Other problem', 'total_chunks': 3}


In [6]:
question = "Are there delays with money transfers?"

result = rag_pipeline.answer(question)

print("Question:", result["question"])
print("Answer:\n", result["answer"])
print("\nSample Source Metadata:", result["sources"][0]["metadata"])


Question: Are there delays with money transfers?
Answer:
 there can be delays but holding onto an account holders money for 3 weeks seems excessive - especially given that the dollar amount is so low. it also seems that there should be a requirement that the bank provides notification of this delay but there has been zero communication coming from capital one.

Sample Source Metadata: {'chunk_index': 2, 'company': 'Coinbase, Inc.', 'complaint_id': '2601401', 'date_received': '2017-08-11', 'issue': 'Money was not available when promised', 'product': 'Money transfer, virtual currency, or money service', 'product_category': 'Money Transfer', 'state': 'NV', 'sub_issue': 'nan', 'total_chunks': 3}


In [7]:
question = "What issues do customers report about personal loans?"

result = rag_pipeline.answer(question)

print("Question:", result["question"])
print("Answer:\n", result["answer"])
print("\nSample Source Metadata:", result["sources"][0]["metadata"])


Question: What issues do customers report about personal loans?
Answer:
 a balance of for a personal loan about their customers or their customer 's money

Sample Source Metadata: {'chunk_index': 0, 'company': 'Self Financial Inc.', 'complaint_id': '10792484', 'date_received': '2024-11-13', 'issue': "Received a loan you didn't apply for", 'product': 'Payday loan, title loan, personal loan, or advance loan', 'product_category': 'Personal Loan', 'state': 'PA', 'sub_issue': 'nan', 'total_chunks': 1}


In [8]:
question = "Do customers complain about unauthorized transactions?"

result = rag_pipeline.answer(question)

print("Question:", result["question"])
print("Answer:\n", result["answer"])
print("\nSample Source Metadata:", result["sources"][0]["metadata"])


Question: Do customers complain about unauthorized transactions?
Answer:
 no

Sample Source Metadata: {'chunk_index': 0, 'company': 'Block, Inc.', 'complaint_id': '11631957', 'date_received': '2025-01-19', 'issue': 'Unauthorized transactions or other transaction problem', 'product': 'Money transfer, virtual currency, or money service', 'product_category': 'Money Transfer', 'state': 'TX', 'sub_issue': 'nan', 'total_chunks': 1}


In [9]:
question = "Are customers unhappy with savings account fees?"

result = rag_pipeline.answer(question)

print("Question:", result["question"])
print("Answer:\n", result["answer"])
print("\nSample Source Metadata:", result["sources"][0]["metadata"])


Question: Are customers unhappy with savings account fees?
Answer:
 yes

Sample Source Metadata: {'chunk_index': 1, 'company': 'WELLS FARGO & COMPANY', 'complaint_id': '2778339', 'date_received': '2018-01-10', 'issue': 'Managing an account', 'product': 'Checking or savings account', 'product_category': 'Savings Account', 'state': 'FL', 'sub_issue': 'Fee problem', 'total_chunks': 2}


In [10]:
# Run all predefined evaluation questions
eval_df = evaluate(rag_pipeline)
eval_df


,Question,Generated Answer,Retrieved Sources,Quality Score (1-5),Comments/Analysis
0,What problems do customers have with credit ca...,lack of service and clarity,me i've run into problems with their credit ca...,2,Output too brief or fragmented. Likely model c...
1,Do customers complain about credit card billing?,I have maintained dozens of credit cards durin...,nd i have learned a lot over this complaint cy...,4,Coherent answer generated from context.
2,Are there issues with credit card fees?,yes,"when signing up for this credit card, the term...",2,Output too brief or fragmented. Likely model c...
3,Are there delays with money transfers?,there can be delays but holding onto an accoun...,o people like myself can make a decision to no...,4,Coherent answer generated from context.
4,Do customers report failed money transfers?,Yes,should be expected with any money transfer sys...,2,Output too brief or fragmented. Likely model c...
5,Are international transfers a problem?,no,"n research on my end, i confirmed that does no...",2,Output too brief or fragmented. Likely model c...
6,What issues do customers report about personal...,a balance of for a personal loan about their c...,of its reporting of a balance of for a persona...,4,Coherent answer generated from context.
7,Do customers complain about loan payments?,"it is unfair to me, as a consumer, for the loa...",to them on a monthly basis in the form of a ch...,4,Coherent answer generated from context.
8,Are loan interest rates a common complaint?,no,i dont know what the interest rate is or anyth...,2,Output too brief or fragmented. Likely model c...
9,Do customers complain about unauthorized trans...,no,unauthorized transactions and no customer serv...,2,Output too brief or fragmented. Likely model c...


In [11]:
def df_to_markdown(df):
    """Convert a DataFrame to a Markdown table string without external libraries."""
    # Header
    md = "| " + " | ".join(df.columns) + " |\n"
    md += "| " + " | ".join(["---"] * len(df.columns)) + " |\n"

    # Rows
    for _, row in df.iterrows():
        md += "| " + " | ".join(str(x) for x in row.values) + " |\n"
    return md

# Usage
print(df_to_markdown(eval_df))


| Question | Generated Answer | Retrieved Sources | Quality Score (1-5) | Comments/Analysis |
| --- | --- | --- | --- | --- |
| What problems do customers have with credit cards? | lack of service and clarity | me i've run into problems with their credit cards. this is just the first time it's impacted my abil... | the lack of service and clarity for its customers. the process i am going through often happens in t... | have in my nearly 30 years of having credit cards. i have never had so many problems in such a short... | 2 | Output too brief or fragmented. Likely model capacity issue. |
| Do customers complain about credit card billing? | I have maintained dozens of credit cards during the last 30 years, and I have never had to complain about the service i was getting from a credit card company | nd i have learned a lot over this complaint cycle. maybe at the least it will help others that have ... | as both a business owner and a head of household, i have maintained dozens of credit